In [1]:
###Python Packages###

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc
from matplotlib.colors import LogNorm
import math as mth

import timeit
import ODE_utils as odeu
from scipy import optimize
import tables
from mpmath import *


In [2]:
###Initial Values###

M0=1.0
Q=0.95
Lambda=0.0000
scalarfield=False
vscalarfield=False


In [3]:
Tol=1
N=50
Nul=100
Nup=10**(40)
scal=10
umax=15
vmax=100
datatype=np.float64


#ru0=5.0
#dr0v=.4
ru0=2.0
#dr0v=0.045
dr0v=1/(4*.375)*(1-2*M0/ru0+Q**2/ru0**2)
#dr0v=0.045

bdytype="stan"
scaltostan=False
solveaffine=False

Elist=[1]
#Elist=[1,2,4,8,16]

#uloc=[1/8,1/4,3/8,1/2,5/8]
#uloc=[1/5,2/5,3/5,4/5,9/10]
uloc=[8.2/10]

In [4]:
###Defining Needed Constants###

scalf=float(scal)
u0=0.0
v0=0.0
rv0=ru0
phiu0=0.0
phiv0=0.0

if M0==0.0:
    dv0=1/N    
else:
    dv0=M0/N
    


def fr(r,M0,Q,Lambda):
    return 1-2*M0/r+Q**2.0/r**2.0-Lambda*r**2.0/3.0

if Lambda>0 or Lambda<0:
    sol = optimize.root(fr,[0.687,1.3,50.0],args=(M0,Q,Lambda), method='hybr')
    rminus=sol.x[0]
    rplus=sol.x[1]
    rcosm=sol.x[2]
else:
    rplus=M0+(M0**2-Q**2)**(.5)
    rminus=M0-(M0**2-Q**2)**(.5)
    rcosm=0.0
    

wHD=True 

print("r+: "+str(rplus))
print("r-: "+str(rminus))
print("rc: "+str(rcosm))

###Predicting Run time###

###timer
start = timeit.default_timer()

Nu=Nul*umax*100#int(umax/du0)
Nv=int(vmax/dv0)
#print("Number of points for lowest iteration is "+str(Nu*Nv)+","+str(Nu)+"X"+str(Nv))
print("Number of points for highest iteration is "+str(Nu*Nv*max(Elist)**2)+","+str(Nu*max(Elist))+"X"+str(Nv*max(Elist)))

numpoints=0.0
for i in range(0,len(Elist)):
    numpoints=Elist[i]**2.0*Nu*Nv+numpoints
print("Total number of points is "+str(int(numpoints)))

predtime=numpoints/(78000)

print("Runtime: about "+str(format(predtime/60,'.2f'))+" minutes")


r+: 1.31224989991992
r-: 0.6877501000800801
rc: 0.0
Number of points for highest iteration is 750000000,150000X5000
Total number of points is 750000000
Runtime: about 160.26 minutes


In [5]:
##################################
###Applying Boundary Conditions###

Emax=max([Elist])


rnpf=np.zeros((2,Nv),dtype=datatype)
signpf=np.zeros((2,Nv),dtype=datatype)
phinpf=np.zeros((2,Nv),dtype=datatype)
drnpvf=np.zeros((2,Nv),dtype=datatype)
dsignpvf=np.zeros((2,Nv),dtype=datatype)
dphinpvf=np.zeros((2,Nv),dtype=datatype)
drnpuf=np.zeros((2,Nv),dtype=datatype)
dsignpuf=np.zeros((2,Nv),dtype=datatype)
dphinpuf=np.zeros((2,Nv),dtype=datatype)
massnpf=np.full((2,Nv),M0)#np.zeros((2,Nv))
#drunpv=np.full((1,Nv-1),-1/(4*dr0v)*(1-2*M0/ru0+Q**2/ru0**2))
 


rnpf[0], signpf[0], phinpf[0] , drnpvf[0], dsignpvf[0], dphinpvf[0], drnpuf[0], dsignpuf[0], dphinpuf[0] = odeu.boundaryv(scal,bdytype,Nv,ru0,dr0v,dv0,vmax,M0,Q,Lambda,scalarfield,datatype)
massnpf[0]=M0

print(dsignpuf[0])

Using Standard Coordinates Along V
[0.         0.00080726 0.00161341 ... 0.44517402 0.44517775 0.44518147]


In [ ]:
drunptemp=np.empty((Nv))*np.nan
#dsigunp=np.empty((Nu))*np.nan


A=.115
u1=9.5
u2=10

###Applying Propagation Algorithm###

urange=np.array([0.0],dtype=datatype)

massnpf[0][0]=1.0

#drunp[0]=-mth.exp(signpf[0][0])/(4.0*dr0v)*(1-2*M0/ru0+Q**2.0/ru0**2.0-Lambda*ru0**2/3)
#dsigunpvalue=100*np.exp(np.nanmax(signpf))
#drunptemp=drunp[0]
    

i=0
dv=dv0
if M0>0:
    du0=M0/Nul
elif M0==0:
    du0=1/Nul

du=du0

###fix this part (put in front of boundary conditions so it is an initial condition)
#if bdytype=="edd" or bdytype=="fulledd":
    #bdyvalue=1.0
    #dsignpuf[0]=2*(3*Q**2-3*M0*ru0+ru0**4*Lambda)/(ru0*(-3*Q**2+ru0*(6*M0-3*ru0+ru0**3*Lambda)))*drunp[0]
    #dsigunp[0]=2.0*(M0*rnpf[0]-Q**2.0)/(rnpf[0]*(Q**2.0+rnpf[0]*(-2*M0+rnpf[0])))*drunp[0]
#else:
    #bdyvalue=0.0
    #dsigunp[0]=0.0
######
    
atom = tables.Float64Atom()

rnpfile = tables.open_file('rnp.h5', mode='w',encoding="utf8") 
rnp = rnpfile.create_earray(rnpfile.root, 'data', atom, (0, Nv))

signpfile = tables.open_file('signp.h5', mode='w',encoding="utf8") 
signp = signpfile.create_earray(signpfile.root, 'data', atom, (0, Nv))
phinpfile = tables.open_file('phinp.h5', mode='w',encoding="utf8") 
phinp = phinpfile.create_earray(phinpfile.root, 'data', atom, (0, Nv))
massnpfile = tables.open_file('massnp.h5', mode='w',encoding="utf8") 
massnp = massnpfile.create_earray(massnpfile.root, 'data', atom, (0, Nv))
    
 
#rtemp=rnpf[0]   
#print(rtemp)
rnp.append(rnpf[0].reshape(1,Nv))
signp.append(signpf[0].reshape(1,Nv))
phinp.append(phinpf[0].reshape(1,Nv))
massnp.append(massnpf[0].reshape(1,Nv))

#drunp=np.append(drunp,np.nan)
#dsigunp=np.append(dsigunp,np.nan)
#dphinpuf=np.append(dphinpuf,np.nan)
drunptemp=np.nanmax(np.abs(drnpuf[0]))     
dsigunpvalue=np.nanmax(np.abs(dsignpuf[0]))
dphiutemp=np.nanmax(np.abs(dphinpuf[0]))

i=0
TempTol=1 
TempTolv=1
Eres=False
while max(urange)<umax and i<50000:
    
    #rnpf=np.insert(rnpf,(1)*Nv,np.empty((Nv))*np.nan).reshape(i+2,Nv)
    #phinpf=np.insert(phinpf,(1)*Nv,np.empty((Nv))*np.nan).reshape(i+2,Nv)
    #signpf=np.insert(signpf,(1)*Nv,np.empty((Nv))*np.nan).reshape(i+2,Nv)
    #massnpf=np.insert(massnpf,(1)*Nv,np.empty((Nv))*np.nan).reshape(i+2,Nv)
    #drunp=np.insert(drunp,(1)*Nv,np.empty((Nv))*np.nan).reshape(i+2,Nv)
    #dsigunp=np.insert(dsigunp,(1)*Nv,np.empty((Nv))*np.nan).reshape(i+2,Nv)
    #dphinpuf=np.insert(dphinpuf,(1),np.empty((1))*np.nan).reshape(i+2)
    
    #drunp=np.append(drunp,np.nan)
    #dsigunp=np.append(dsigunp,np.nan)
    #dphinpuf=np.append(dphinpuf,np.nan)
    
    du=du0*Tol*TempTol/max([abs(dsigunpvalue),abs(drunptemp),abs(dphiutemp)])
    #du=abs(du0*Tol*TempTol/mth.exp(np.nanmax(signpf[0,:])))
    #dui=abs(mth.exp(np.nanmax(signpf[0,:]))/(du0*Tol))
    
    if du<1/Nup*1.1:
        du=1/Nup
     
    
    if du>=1/Nul:
        #print('r='+str(rnpf[i][0]))
        if rnpf[1][0]<rplus:
            du=1/(1*Nul)
            #print(rplus)
        else:
            du=1/Nul
    
    
    
    
    print("%.24f" % urange[i])
    print("%.24f" % du)
    
    ###fix this section (dphinpuf is in the wrong form)
    #if urange[i]>u1 and urange[i]<u2 and vscalarfield==True:
        #u=urange[i]+du
        #phinpf[1][0]=A*64*(u-u1)**3.0*(u2-u)**3.0/(u2-u1)**6.0
        #dphinpuf[i]=192*A*(u-u1)**2.0*(u-u2)**2.0*(-2*u+u1+u2)/(u1-u2)**6.0 
    #else:
        #continue
        #phinpf[1][0]=0.0
        #dphinpuf[i+1]=0.0
    
    #print(drunp,dsigunp,dphinpuf)
    
    
    #rnpf[1][0]=rnpf[0][0]+du*drunp[i]
    #drunp[i+1]=drunp[i]+du*(drunp[i]*dsigunp[i]-rnpf[0][0]*dphinpuf[i]**2.0)
    
    #signpf[1][0]=(signpf[0][0]+du*dsigunp[i])*(1.0-bdyvalue)+np.log(abs(1.0-2.0*M0/rnpf[1][0]+Q**2.0/rnpf[1][0]**2.0))*bdyvalue
    #dsigunp[i+1]=2*(3*Q**2-3*M0*rnpf[1][0]+rnpf[1][0]**4*Lambda)/(rnpf[1][0]*(-3*Q**2+rnpf[1][0]*(6*M0-3*rnpf[1][0]+rnpf[1][0]**3*Lambda)))*drunp[i+1]*bdyvalue

    #massnpf[1][0]=M0
    ###Predictor###
    drnpvf[1]=drnpvf[0]+du*odeu.Rfunc(drnpvf[0],drnpuf[0],rnpf[0],signpf[0],Q)
    dsignpvf[1]=dsignpvf[0]+du*odeu.Sigfunc(drnpvf[0],drnpuf[0],dphinpuf[0],dphinpvf[0],rnpf[0],signpf[0],Q)
    dphinpvf[1]=dphinpvf[0]+du*odeu.Phifunc(drnpvf[0],drnpuf[0],dphinpuf[0],dphinpvf[0],rnpf[0])
    drnpuf[1]=drnpuf[0]+du*odeu.Coneq(drnpuf[0],dsignpuf[0],dphinpuf[0],rnpf[0])#(drnpuf[0]*dsignpuf[0]-rnpf[0]*np.power(dphinpuf[0],2.0))
    
    rnpf[1]=rnpf[0]+du*drnpuf[0]
    signpf[1]=signpf[0]+du*dsignpuf[0]
    phinpf[1]=phinpf[0]+du*dphinpuf[0]
    
    dsignpuf[1][0]=0.0#-1/(4*dsignpvf[1][0])*np.exp()
    dphinpuf[1][0]=0.0
    signpf[1][0]=0.0
    phinpf[1][0]=0.0
        
    for j in range(0,Nv-1):
        dphinpuf[1][j+1]=dphinpuf[1][j]+dv*odeu.Phifunc(drnpvf[1][j],drnpuf[1][j],dphinpuf[1][j],dphinpvf[1][j],rnpf[1][j])         
        dsignpuf[1][j+1]=dsignpuf[1][j]+dv*odeu.Sigfunc(drnpvf[1][j],drnpuf[1][j],dphinpuf[1][j],dphinpvf[1][j], rnpf[1][j],signpf[1][j],Q)                                                                             
        #signpf[1][i+1]=signpf[0][i]+du*dsignpuf[0][i+1]
    #dsignpuf[1]=???
    #dphinpuf[1]=???
    #rnpf[1]=rnpf[0]+du*drnpuf[0]
    #signpf[1]=signpf[0]+du*dsignpuf[0]
    #phinpf[1]=phinpf[0]+du*dphinpuf[0]
    
    ###Correction###
    drnpvf[1]=drnpvf[0]+1/2*du*(odeu.Rfunc(drnpvf[0],drnpuf[0],rnpf[0],signpf[0],Q)+odeu.Rfunc(drnpvf[1],drnpuf[1],rnpf[1],signpf[1],Q))
    dsignpvf[1]=dsignpvf[0]+1/2*du*(odeu.Sigfunc(drnpvf[0],drnpuf[0],dphinpuf[0],dphinpvf[0],rnpf[0],signpf[0],Q)+odeu.Sigfunc(drnpvf[1],drnpuf[1],dphinpuf[1],dphinpvf[1],rnpf[1],signpf[1],Q))
    dphinpvf[1]=dphinpvf[0]+1/2*du*(odeu.Phifunc(drnpvf[0],drnpuf[0],dphinpuf[0],dphinpvf[0],rnpf[0])+odeu.Phifunc(drnpvf[1],drnpuf[1],dphinpuf[1],dphinpvf[1],rnpf[1]))
    drnpuf[1]=drnpuf[0]+1/2*du*(odeu.Coneq(drnpuf[0],dsignpuf[0],dphinpuf[0],rnpf[0])+odeu.Coneq(drnpuf[1],dsignpuf[1],dphinpuf[1],rnpf[1]))
    
    rnpf[1]=rnpf[0]+1/2*du*(drnpuf[0]+drnpuf[1])
    signpf[1]=signpf[0]+1/2*du*(dsignpuf[0]+dsignpuf[1])
    phinpf[1]=phinpf[0]+1/2*du*(dphinpuf[0]+dphinpuf[1])
    
    dsignpuf[1][0]=0.0#-1/(4*dsignpvf[1][0])*np.exp()
    dphinpuf[1][0]=0.0
    #signpf[1][0]=0.0
    #phinpf[1][0]=0.0
        
    for j in range(0,Nv-1):
        dphinpuf[1][j+1]=dphinpuf[1][j]+1/2*dv*(odeu.Phifunc(drnpvf[1][j],drnpuf[1][j],dphinpuf[1][j],dphinpvf[1][j],rnpf[1][j])+odeu.Phifunc(drnpvf[1][j+1],drnpuf[1][j+1],dphinpuf[1][j+1],dphinpvf[1][j+1],rnpf[1][j+1]))         
        dsignpuf[1][j+1]=dsignpuf[1][j]+1/2*dv*(odeu.Sigfunc(drnpvf[1][j],drnpuf[1][j],dphinpuf[1][j],dphinpvf[1][j],rnpf[1][j],signpf[1][j],Q)+odeu.Sigfunc(drnpvf[1][j+1],drnpuf[1][j+1],dphinpuf[1][j+1],dphinpvf[1][j+1],rnpf[1][j+1],signpf[1][j+1],Q))    
    massnpf[1]=(1+4.0*np.exp(-signpf[1])*drnpuf[1]*drnpvf[1])*rnpf[1]/2.0+np.power(Q,2.0)/(2.0*rnpf[1])
    
    drunptemp=np.nanmax(np.abs(drnpuf[1]))      
    dsigunpvalue=np.nanmax(np.abs(dsignpuf[1]))    
    dphiutemp=np.nanmax(np.abs(dphinpuf[1]))
    #if any(abs((np.exp(signpf[0][:])-np.exp(signpf[1][:]))/np.exp(signpf[0][:]))>0.01):  
    if float(dsigunpvalue*du)>1000:#0.1:
        TempTol=TempTol/2
        Eres=True
    else:
        rnp.append(rnpf[1].reshape(1,Nv))
        signp.append(signpf[1].reshape(1,Nv))
        phinp.append(phinpf[1].reshape(1,Nv))
        massnp.append(massnpf[1].reshape(1,Nv))
    
        #drunp=np.append(drunp,np.nan)
        #dsigunp=np.append(dsigunp,np.nan)
        #dphinpuf=np.append(dphinpuf,np.nan)
    
        rnpf[0]=rnpf[1]
        phinpf[0]=phinpf[1]
        signpf[0]=signpf[1]
        drnpvf[0]=drnpvf[1]
        dsignpvf[0]=dsignpvf[1]
        dphinpvf[0]=dphinpvf[1]
        drnpuf[0]=drnpuf[1]
        dsignpuf[0]=dsignpuf[1]
        dphinpuf[0]=dphinpuf[1]
        massnpf[0]=massnpf[1]
    
        urange=np.append(urange,urange[i]+du)
        
        if Eres==True:
            TempTol=2*TempTol
        else:
            TempTol=1
        Eres=False
        i+=1
    
    #print(TempTol)
    print(drnpuf[1][-1])
    print(dsignpuf[1][-1])
    print(massnpf[1][-1])
    #print(dphinpuf[1][-1])
    print('---')
    
    #drunp=np.insert(drunp,(1)*Nv,np.empty((Nv))*np.nan).reshape(i+2,Nv)
    #dsigunp=np.insert(dsigunp,(1)*Nv,np.empty((Nv))*np.nan).reshape(i+2,Nv)
    #dphinpuf=np.insert(dphinpuf,(1),np.empty((1))*np.nan).reshape(i+2)
    #print(urange)
     
    #print(urange)
  
    
    #du=abs(du0*Tol/np.nanmin(drunptemp))

    
    
    
    
    if rnpf[1][0]<0.0 or np.isnan(du):
        break
   
    #dumaxlist=[]
    
    
    #if i>1000:
        #break


0.000000000000000000000000
0.006792901364182422560556
-1.4765958925398945
0.4472403769283036
1.0000000438815675
---
0.006792901364182422560556
0.006772333615799910368216
-1.4810874531785216
0.4493085684593295
1.0000000870834738
---
0.013565234979982333796134
0.006751795769074453966441
-1.4855997872596247
0.4513861720139013
1.0000001304563069
---
0.020317030749056788629936
0.006731287986010185686592
-1.4901329892951416
0.4534732256760086
1.000000174000083
---
0.027048318735066974316528
0.006710810425538039913906
-1.4946871541792586
0.4555697676635886
1.0000002177148228
---
0.033759129160605014230434
0.006690363245605772217484
-1.4992623771897555
0.4576758363291701
1.0000002616005266
---
0.040449492406210783845832
0.006669946603171741233984
-1.5038587539893573
0.4597914701605121
1.000000305657205
---
0.047119439009382521610370
0.006649560654198758205546
-1.5084763806270922
0.461916707781237
1.0000003498848504
---
0.053768999663581278081192
0.006629205553648030192770
-1.513115353539656
0.

-1.852910446513699
0.6197868060137046
1.0000035319472196
---
0.438451176582347890153812
0.005396914901535187700232
-1.859135029912922
0.6226262400284427
1.000003587032362
---
0.443848091483883100405450
0.005378845451838094871011
-1.865388130863853
0.625478165004541
1.0000036422768328
---
0.449226936935721188337567
0.005360814639347493554267
-1.8716698747848766
0.6283426304610641
1.00000369768029
---
0.454587751575068699239068
0.005342822542970814686525
-1.877980387591513
0.6312196861114855
1.0000037532423838
---
0.459930574118039503517252
0.005324869240421023670884
-1.8843197956983726
0.6341093818647716
1.000003808962765
---
0.465255443358460529790221
0.005306954808217024567585
-1.8906882260211193
0.6370117678265059
1.0000038648410767
---
0.470562398166677553490445
0.005289079321684155357564
-1.8970858059784481
0.639926894299951
1.0000039208769558
---
0.475851477488361707113285
0.005271242854954767340092
-1.9035126634940707
0.6428548117872029
1.0000039770700369
---
0.481122720343316490

-2.381991867471356
0.8593226156997492
1.0000078805971593
---
0.783212513487404282663817
0.004198167145975889406884
-2.390622107592567
0.8632007832877078
1.0000079460369606
---
0.787410680633380155590828
0.004183011596956376192125
-2.3992912948553524
0.8670955763235398
1.0000080116016254
---
0.791593692230336487547504
0.004167897420976921125935
-2.4079995961919978
0.8710070615282224
1.0000080772905628
---
0.795761589651313383519948
0.004152824616671018341518
-2.4167471792048225
0.8749353059136317
1.000008143103187
---
0.799914414267984374973253
0.004137793181697342788372
-2.425534212169103
0.8788803767841633
1.0000082090389057
---
0.804052207449681755058180
0.004122803112744889350583
-2.4343608640360146
0.882842341738324
1.0000082750971242
---
0.808175010562426687776849
0.004107854405538150129040
-2.4432273044355854
0.8868212686703619
1.0000083412772494
---
0.812282864967964868263550
0.004092947054842332489655
-2.4521337036796713
0.890817225771884
1.0000084075786824
---
0.81637581202280

-3.113606750891755
1.1853474274328775
1.0000128748483328
---
1.049196256957493966766037
0.003211709377600733324870
-3.1255110699545305
1.1906122684840408
1.0000129476962347
---
1.052407966335094702259312
0.003199476749940123598837
-3.1374682598816825
1.1958993857160327
1.0000130206238256
---
1.055607443085034846674830
0.003187283239760045107369
-3.1494785443588977
1.2012088713405356
1.0000130936304656
---
1.058794726324794988059352
0.003175128790101214206615
-3.1615421479980794
1.2065408179950639
1.0000131667155194
---
1.061969855114896166270455
0.003163013343450790920003
-3.173659296341628
1.2118953187451011
1.0000132398783512
---
1.065132868458346981910267
0.003150936841748356361664
-3.18583021586674
1.2172724670863186
1.0000133131183238
---
1.068283805300095368195912
0.003138899226391884088316
-3.1980551339897336
1.2226723569467646
1.0000133864348038
---
1.071422704526487246212696
0.003126900438243696272317
-3.2103342790703904
1.2280950826889874
1.0000134598271542
---
1.074549604964

-4.120212562015624
1.627323293822823
1.0000182864831575
---
1.251440271079107269258657
0.002427059247425808214543
-4.136555285217685
1.6344554717839623
1.0000183634974273
---
1.253867330326533080508966
0.002417470409675367103153
-4.152969630997813
1.6416177926764124
1.0000184405483052
---
1.256284800736208406846117
0.002407915513120993233204
-4.169455902063672
1.648810385489229
1.000018517635251
---
1.258692716249329324185169
0.002398394475176125750782
-4.18601440241868
1.6560333798188063
1.000018594757723
---
1.261091110724505526263783
0.002388907213081254039821
-4.202645437368109
1.6632869058718016
1.0000186719151838
---
1.263480017937586685761175
0.002379453643908267107543
-4.219349313525222
1.670571094468057
1.0000187491071
---
1.265859471581495032665998
0.002370033684564766540692
-4.236126338817426
1.677886077043592
1.0000188263329328
---
1.268229505266059709001070
0.002360647251798358645103
-4.25297682249247
1.6852319856535392
1.0000189035921545
---
1.270590152517858006930851
0.0

...

-1.1527236296435321e-20
-91.42060910184031
1.000083697252966
---
5.045220998075035545582523
0.000109384526073986736053
-1.1355303266510668e-20
-91.56259036897032
1.0000836978482088
---
5.045330382601109775464465
0.000109214909273568387335
-1.118593488793662e-20
-91.70462416025626
1.0000836984434396
---
5.045439597510383400447154
0.000109045755233942574152
-1.1019092901356032e-20
-91.846710434238
1.000083699038659
---
5.045548643265616917119587
0.000108877062147587458677
-1.0854739618224937e-20
-91.98884914953666
1.0000836996338665
---
5.045657520327764267165094
0.000108708828216168298616
-1.0692837912295166e-20
-92.13104026485686
1.0000837002290623
---
5.045766229155980830967110
0.000108541051650476948719
-1.0533351211224095e-20
-92.27328373898351
1.0000837008242465
---
5.045874770207631421214955
0.000108373730670378341854
-1.0376243488309594e-20
-92.4155795307846
1.000083701419419
---
5.045983143938301829223292
0.000108206863504750248027
-1.0221479254348328e-20
-92.55792759920892
1.00

-4.026163924592976e-21
-101.48394653049749
1.000083738892199
---
5.052492068666160740519899
0.000098537752441415417284
-3.9661169167251355e-21
-101.629508639131
1.0000837394866453
---
5.052590606418601915095223
0.000098396618599311434718
-3.9069655232460115e-21
-101.77512055501336
1.0000837400810803
---
5.052689003037201587176241
0.000098255840380897575305
-3.848696384821609e-21
-101.92078224186521
1.0000837406755045
---
5.052787258877582843297205
0.000098115416503273037937
-3.791296341408989e-21
-102.06649366347311
1.0000837412699175
---
5.052885374294086240354318
0.000097975345689559385103
-3.734752429282985e-21
-102.2122547836904
1.0000837418643194
---
5.052983349639776022854676
0.000097835626668864560931
-3.67905187810728e-21
-102.35806556643607
1.0000837424587106
---
5.053081185266444563808363
0.000097696258176249389338
-3.6241821080491936e-21
-102.50392597569423
1.0000837430530904
---
5.053178881524621246512652
0.000097557238952693421663
-3.5701307269375136e-21
-102.6498359755149

-1.4063432158300216e-21
-111.7916511960621
1.0000837804771872
---
5.059067232254357726617400
0.000089452118230741843878
-1.3853701511134217e-21
-111.94061565441537
1.0000837810708791
---
5.059156684372588586029451
0.000089333080236686740559
-1.3647098802187508e-21
-112.08962753046194
1.0000837816645602
---
5.059246017452824872862038
0.000089214320899428086538
-1.3443577378031278e-21
-112.23868679212907
1.0000837822582311
---
5.059335231773724572690298
0.000089095839285080331003
-1.3243091281129324e-21
-112.38779340739801
1.0000837828518914
---
5.059424327613009531035004
0.000088977634463830864125
-1.304559523945705e-21
-112.53694734430437
1.0000837834455414
---
5.059513305247473446968343
0.000088859705509917696052
-1.2851044656275354e-21
-112.68614857093688
1.0000837840391807
---
5.059602164952983649470752
0.000088742051501608613115
-1.2659395600057056e-21
-112.8353970554394
1.0000837846328094
---
5.059690907004485538323024
0.000088624671521177894595
-1.2470604794563633e-21
-112.984692

-4.91269928463816e-22
-122.33196940184754
1.0000838220105839
---
5.065055847755567164369950
0.000081744780607194032426
-4.839439555916978e-22
-122.4841784857096
1.0000838226035553
---
5.065137592536174793167447
0.000081643197706279371285
-4.767272355839111e-22
-122.63643286568178
1.0000838231965168
---
5.065219235733881220085095
0.000081541836845112368785
-4.696181390485439e-22
-122.7887325131631
1.000083823789468
---
5.065300777570726609155827
0.000081440697328869228445
-4.626150608959518e-22
-122.94107739959846
1.000083824382409
---
5.065382218268055680709949
0.000081339778465555094751
-4.557164199762645e-22
-123.0934674964774
1.0000838249753403
---
5.065463558046521264088824
0.000081239079565990554837
-4.489206587222989e-22
-123.24590277533449
1.0000838255682614
---
5.065544797126086962180125
0.000081138599943797286361
-4.422262427978006e-22
-123.39838320774936
1.0000838261611724
---
5.065625935726030704131517
0.000081038338915383813796
-4.356316607509305e-22
-123.55090876534607
1.0

-1.716225288467196e-22
-133.09446986570495
1.0000838634945373
---
5.070544209547540148719236
0.000075134601836501590079
-1.69063370604498e-22
-133.24978297929
1.000083864086816
---
5.070619344149376317432143
0.000075047026542281313898
-1.6654237511433638e-22
-133.40513949011827
1.0000838646790848
---
5.070694391175918802616707
0.000074959630777498876012
-1.6405897325572073e-22
-133.56053937249462
1.000083865271344
---
5.070769350806695996425333
0.000074872414015268598866
-1.6161260439590532e-22
-133.71598260076348
1.0000838658635935
---
5.070844223220711377564385
0.000074785375730716149010
-1.5920271626331948e-22
-133.87146914930557
1.0000838664558334
---
5.070919008596441734937343
0.000074698515400970879924
-1.5682876482286245e-22
-134.02699899254077
1.0000838670480636
---
5.070993707111842496715326
0.000074611832505154773157
-1.5449021415305856e-22
-134.18257210492584
1.000083867640284
---
5.071068318944347730337086
0.000074525326524374320360
-1.5218653632504442e-22
-134.338188460955

In [ ]:
rnpfile.close()
signpfile.close()
phinpfile.close()
massnpfile.close()

In [ ]:
Nu=i+2

print(Nu)
#rnpf[i+1][j+1]=answer[0]
#phinpf[i+1][j+1]=answer[1]
#signpf[i+1][j+1]=answer[2]
#massnpf[i+1][j+1]=answer[3]
#drunp[i+1][j+1]=answer[4]
#dsigunp[i+1][j+1]=answer[5]

In [ ]:
#print(rnpf[1,0])

In [ ]:
#print(dnu.x4giveralt(0,0,du,dv,rnpf,phinpf,signpf,Q,Lambda))

In [ ]:
#print(rnpf)

In [ ]:
vrange=np.arange(0,vmax,dv0)
#print(urange)

In [ ]:
switch=0.0
np.save('rnputil',np.array([Nu*Nv,Nu,Nv,ru0,dr0v,M0,Q,Lambda,wHD,switch]))
np.save('urange',urange)
np.save('vrange',vrange)

In [ ]:
stop = timeit.default_timer()

In [ ]:
acttime=stop - start

numpoints=Nu*Nv



print(str(acttime)+' seconds')
print(str((acttime)/60)+' minutes')
print(str((acttime)/3600)+' hours')

print(str(numpoints)+' points')

print(str((numpoints)/(stop - start))+' points per second') 

exttimef=(acttime-predtime)/60

exttime=format(abs((acttime-predtime)/60),'.2f')
if exttimef>0.0:
    print("Took "+str(exttime)+" more minutes")
if exttimef<0.0:
    print("Took "+str(exttime)+" less minutes")
if exttimef==0.0:
    print("Took exactly the right time!!!!")

conv=False
j=0
tempanswer=[]
    while conv==False:
        dv=TempTolv*dv0
        jcount=1
        while j<Nv-1:
            j1=mth.ceil(jcount*TempTolv)
            j2=mth.floor(jcount*TempTolv)
            answer=dnu.x4giveralt(0,j+mth.ceil((jcount-1)*TempTolv),du,dv*TempTolv,rnpf,phinpf,signpf,massnpf,M0,Q,Lambda,datatype)
            rnpf[1][j+j1]=answer[0]
            phinpf[1][j+j1]=answer[1]
            signpf[1][j+j1]=answer[2]
            massnpf[1][j+j1]=answer[3]
            #drunptemp[j+1]=answer[4]
            dsigunpv=answer[5]
            jcount=(jcount+1)**(0**j2)
            j=j+j2
        
        if np.abs(tempanswer[0]-answer[0])/answer[0]>0.01 or not tempanswer:
            tempanswer=answer
            TempTol=TempTolv/2
            
        else:
            conv=True
            